# Parallel reduction

<a href="https://colab.research.google.com/github/mark-hobbs/articles/blob/main/cuda/parallel-reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See [this post](https://github.com/googlecolab/colabtools/issues/5081) to understand compatability issues with Google Colab and Numba CUDA

Literature:
- [Optimising parallel reduction in CUDA](https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf)

## Reduce bond forces to particle forces

Bond forces can be stored as a bondlist or neighbour list

- bondlist [n_bonds, 2]
- neighbourlist [n_particles, n_family_members]

Reduce:
- particles.forces [n_particles, 1]

In [64]:
!uv pip install -q --system numba-cuda==0.15.0

import numpy as np
from numba import  njit, prange, config
config.CUDA_ENABLE_PYNVJITLINK = 1

try:
    import google.colab
    !git clone https://github.com/mark-hobbs/articles.git
    import os
    os.chdir('articles/cuda')  # Navigate to the cuda subdirectory
except ImportError:
    pass  # Already local, no need to clone

import utils

Cloning into 'articles'...
remote: Enumerating objects: 567, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 567 (delta 53), reused 47 (delta 11), pack-reused 464 (from 1)
Receiving objects: 100% (567/567), 101.08 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (267/267), done.


In [65]:
np.random.seed(42)
n_particles = 1500000
n_family_members = 128

bond_forces = np.random.rand(n_particles, n_family_members)

### Numpy and Numba

In [66]:
@utils.profile(runs=10)
def reduce_bond_forces_a(bond_forces):
    n_particles = bond_forces.shape[0]
    f = np.zeros((n_particles))
    for i in range(n_particles):
        f[i] = np.sum(bond_forces[i, :])
    return f

In [67]:
@utils.profile(runs=10)
def reduce_bond_forces_b(bond_forces):
    return np.sum(bond_forces, axis=1)

In [68]:
@utils.profile(runs=10)
@njit(parallel=True, fastmath=True)
def reduce_bond_forces_c(bond_forces):
    n_particles = bond_forces.shape[0]
    f = np.zeros((n_particles))
    for i in prange(n_particles):
        f[i] = np.sum(bond_forces[i, :])
    return f

In [69]:
f_a = reduce_bond_forces_a(bond_forces)
f_b = reduce_bond_forces_b(bond_forces)
f_c = reduce_bond_forces_c(bond_forces)
assert np.allclose(f_a, f_b) and np.allclose(f_b, f_c), "Results are not equal"

Function 'reduce_bond_forces_a' executed 10 time(s)
Average execution time: 5.0456 seconds
Min: 4.9510s, Max: 5.1096s

Function 'reduce_bond_forces_b' executed 10 time(s)
Average execution time: 0.1263 seconds
Min: 0.1259s, Max: 0.1266s

Function 'reduce_bond_forces_c' executed 10 time(s)
Average execution time: 0.0817 seconds
Min: 0.0214s, Max: 0.6228s



### Numba CUDA

In [70]:
from numba import cuda, float32

In [71]:
THREADS_PER_BLOCK = 128

@cuda.jit
def reduce_bond_forces_kernel(bond_forces, particle_forces):
    """
    Reduce bond forces to particle forces

    Employ sequential addressing
    """

    shared = cuda.shared.array(THREADS_PER_BLOCK, dtype=bond_forces.dtype)

    particle = cuda.blockIdx.x
    tid = cuda.threadIdx.x
    n_family_members = bond_forces.shape[1]

    # Initialise shared memory
    val = 0.0
    if tid < n_family_members:
        val = bond_forces[particle, tid]
    shared[tid] = val

    cuda.syncthreads()

    stride = THREADS_PER_BLOCK // 2
    while stride > 0:
        if tid < stride:
            shared[tid] += shared[tid + stride]
        cuda.syncthreads()
        stride //= 2

    if tid == 0:
        particle_forces[particle] = shared[0]

@utils.profile(runs=10)
def reduce_bond_forces_gpu(bond_forces):
    n_particles, n_family_members = bond_forces.shape

    bond_forces = cuda.to_device(bond_forces.astype(np.float32))
    f = cuda.device_array(n_particles, dtype=np.float32)

    reduce_bond_forces_kernel[n_particles, THREADS_PER_BLOCK](bond_forces, f)
    return f.copy_to_host()

In [72]:
utils.get_cuda_device_info()

CUDA Device Information:
----------------------------------------
CUDA Runtime Version:          12.6
Device Name:                   NVIDIA L4                                                                                                                       
Compute Capability:            (8, 9)

Memory:
Total Memory:                  23.80 GB
Free Memory:                   19.71 GB

Compute Resources:
Streaming Multiprocessors:     58
Max Threads per Block:         1024

Grid Limitations:
Max Grid Dimensions X:         2147483647
Max Grid Dimensions Y:         65535
Max Grid Dimensions Z:         65535

Additional Characteristics:
Warp Size:                     32
Clock Rate:                    2.04 GHz
Memory Clock Rate:             6.25 GHz


In [73]:
f_gpu = reduce_bond_forces_gpu(bond_forces)
assert np.allclose(f_a, f_gpu), "Results are not equal"

Function 'reduce_bond_forces_gpu' executed 10 time(s)
Average execution time: 0.4640 seconds
Min: 0.4489s, Max: 0.5657s



In [74]:
import time

def benchmark_kernel(bond_forces, num_runs=100):
    n_particles, n_family_members = bond_forces.shape

    bond_forces = cuda.to_device(bond_forces.astype(np.float32))
    f = cuda.device_array(n_particles, dtype=np.float32)

    # Warm up the kernel
    for _ in range(5):
        reduce_bond_forces_kernel[n_particles, THREADS_PER_BLOCK](bond_forces, f)

    cuda.synchronize()

    start = time.perf_counter()

    for _ in range(num_runs):
        reduce_bond_forces_kernel[n_particles, THREADS_PER_BLOCK](bond_forces, f)

    cuda.synchronize()

    end = time.perf_counter()

    avg = (end - start) / num_runs
    return avg, f.copy_to_host()

In [75]:
cuda_event_time, result = benchmark_kernel(bond_forces, num_runs=100)
print(f"Kernel executed in {cuda_event_time:.4f} seconds")
assert np.allclose(f_a, result), "Results are not equal"

Kernel executed in 0.0041 seconds


In [76]:
print(f_a, "\n", result)

[61.7185851  63.45910746 62.32717021 ... 67.28637261 67.38296414
 64.04946335] 
 [61.718586 63.459106 62.32717  ... 67.28638  67.382965 64.04947 ]


## First add during load

In [77]:
THREADS_PER_BLOCK = 128

@cuda.jit
def reduce_bond_forces_kernel(bond_forces, particle_forces):
    """
    Reduce bond forces to particle forces

    Employ sequential addressing with optimised first add during load
    """

    shared = cuda.shared.array(THREADS_PER_BLOCK, dtype=bond_forces.dtype)

    particle = cuda.blockIdx.x
    tid = cuda.threadIdx.x
    n_family_members = bond_forces.shape[1]

    val = 0.0
    if tid < n_family_members:
        val += bond_forces[particle, tid]

    if tid + THREADS_PER_BLOCK // 2 < n_family_members:
        val += bond_forces[particle, tid + THREADS_PER_BLOCK // 2]

    shared[tid] = val
    cuda.syncthreads()

    stride = THREADS_PER_BLOCK // 4  # Start with quarter instead of half
    while stride > 0:
        if tid < stride:
            shared[tid] += shared[tid + stride]
        cuda.syncthreads()
        stride //= 2

    if tid == 0:
        particle_forces[particle] = shared[0]

In [78]:
cuda_event_time, result = benchmark_kernel(bond_forces, num_runs=100)
print(f"Kernel executed in {cuda_event_time:.4f} seconds")
assert np.allclose(f_a, result), "Results are not equal"

Kernel executed in 0.0062 seconds


In [79]:
print(f_a, "\n", result)

[61.7185851  63.45910746 62.32717021 ... 67.28637261 67.38296414
 64.04946335] 
 [61.718586 63.459106 62.32717  ... 67.28638  67.382965 64.04947 ]
